# Code to measure the offset of each MIRI galaxy to the centre of the frame

To do this, we will read in the catalogue of galaxies to obtain their respective IDs, RAs and Decs. Then we will fit a centroid to all our MIRI images and determine their coordinates. From this we can calculate the spherical offsets. Lastly we will plot the results and see what happens.

The code is separated into several functions that are being called by one main function for better readability and easier debugging.

In [ ]:
from astropy.io import fits
from astropy.table import Table
import numpy as np
from matplotlib import pyplot as plt
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
import astropy.units as u
from astropy.nddata import Cutout2D
from photutils import centroids
import pandas as pd
import scipy
import os
import glob

Define the necessary functions:

In [ ]:

def load_cutout(file_path, index=1):
    """Loads a FITS cutout file and extracts the data, header, and WCS."""
    try:
        with fits.open(file_path) as hdu:
            data = hdu[index].data
            header = hdu[index].header
            wcs = WCS(header)
        return data, wcs
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return None, None

def compute_centroid(cutout, smooth_sigma, good_frac_cutout, smooth_miri):
    """Computes the centroid using a smoothed version of the image."""
    if cutout is None:
        return None

    # Decide whether to smooth MIRI or not
    if smooth_miri == True:
        smoothed_data = scipy.ndimage.gaussian_filter(cutout.data, smooth_sigma)
    else: 
        smoothed_data = cutout.data
    
    # Makes sure the boxsize is an odd number
    search_boxsize = int(np.floor(good_frac_cutout * cutout.shape[0]) // 2 * 2 + 1)

    centroid_pix = centroids.centroid_quadratic(
        smoothed_data,
        xpeak=cutout.shape[0] // 2,
        ypeak=cutout.shape[1] // 2,
        search_boxsize=search_boxsize,
        fit_boxsize=5
    )

    return cutout.wcs.pixel_to_world(centroid_pix[0], centroid_pix[1]) if not np.isnan(centroid_pix).any() else None

def save_alignment_figure(g, cutout_nircam, cutout_miri, centroid_nircam, centroid_miri, output_dir, survey, filter):
    """Saves a figure showing centroid alignment."""
    fig, axs = plt.subplots(1, 2, figsize=[10, 5])

    axs[0].imshow(scipy.ndimage.gaussian_filter(cutout_nircam.data, 1.0), origin='lower')
    axs[0].plot(*cutout_nircam.wcs.world_to_pixel(centroid_nircam), 'x', color='red')
    axs[0].set(title=f"NIRCam F444W Reference {g['id']}")

    axs[1].imshow(scipy.ndimage.gaussian_filter(cutout_miri.data, 1.0), origin='lower')
    axs[1].plot(*cutout_miri.wcs.world_to_pixel(centroid_miri), 'o', color='orange')
    axs[1].set(title=f"MIRI {filter} Cutout {g['id']}")
    
    # show expected position of the centroid
    expected_position_pix = cutout_miri.wcs.world_to_pixel(centroid_nircam)
    axs[1].plot(expected_position_pix[0], expected_position_pix[1], 'x', color='red')

    output_path = os.path.join(output_dir, f"{g['id']}_{filter}_offset_{survey}.pdf")
    os.makedirs(output_dir, exist_ok=True)
    fig.savefig(output_path)
    plt.close()

    #print(f"Saved figure: {output_path}")


def compute_offset(cutout_folder, survey, filter, obs="", smooth_miri=True):
    """Computes the astrometric offset between NIRCam and MIRI for each galaxy."""

    global global_cat
    
    for i, g in enumerate(global_cat):
        #print(f"Processing galaxy {g['id']}...")

        ref_position = SkyCoord(ra=g['ra'], dec=g['dec'], unit=u.deg)
        cutout_size = (2.5 * u.arcsec, 2.5 * u.arcsec)
        smooth_sigma, good_frac_cutout = 1.0, 0.7

        if g['id'] == 21451: # exclude bright source nearby
            good_frac_cutout = 0.4
        if g['id'] == 9986: # exclude bright source nearby
            good_frac_cutout = 0.4
        if g['id'] == 11451: # exclude bright source nearby
            good_frac_cutout = 0.4
        
        # Load MIRI cutout
        cutout_miri_path = os.path.join(cutout_folder, f"{g['id']}_{filter}_cutout_{survey}{obs}_rot.fits")
        miri_data, miri_wcs = load_cutout(cutout_miri_path)
        if miri_data is None:
            continue
        cutout_miri = Cutout2D(miri_data, ref_position, cutout_size, wcs=miri_wcs)

        # Load NIRCam cutout
        nircam_path = f"/home/bpc/University/master/Red_Cardinal/NIRCam/F444W_cutouts/{g['id']}_F444W_cutout.fits"
        nircam_data, nircam_wcs = load_cutout(nircam_path)
        if nircam_data is None:
            continue
        cutout_nircam = Cutout2D(nircam_data, ref_position, cutout_size, wcs=nircam_wcs)

        # Compute centroids
        centroid_nircam = compute_centroid(cutout_nircam, smooth_sigma, good_frac_cutout, smooth_miri)
        centroid_miri = compute_centroid(cutout_miri, smooth_sigma, good_frac_cutout, smooth_miri)

        if centroid_nircam is None or centroid_miri is None:
            print("Centroid not found for one or both cutouts. Skipping.")
            continue

        # Save alignment figure
        output_dir = f"/home/bpc/University/master/Red_Cardinal/offsets/{survey}{obs}/"
        save_alignment_figure(g, cutout_nircam, cutout_miri, centroid_nircam, centroid_miri, output_dir, survey, filter)

        # Compute offsets
        dra, ddec = centroid_nircam.spherical_offsets_to(centroid_miri)
        global_cat[f'{survey}{obs}_dra'][i] = dra.to(u.arcsec).value
        global_cat[f'{survey}{obs}_ddec'][i] = ddec.to(u.arcsec).value

        #print(f"Offset: ΔRA = {dra.to(u.arcsec)}, ΔDec = {ddec.to(u.arcsec)}")



Define the global catalogue:

In [ ]:
catalogue = '/home/bpc/University/master/Red_Cardinal/cat_targets.fits'

global_cat = Table.read(catalogue)

# Add the columns to store the astrometric offsets
for col in ["primer003_dra", "primer003_ddec", 
            "primer004_dra", "primer004_ddec", 
            "cweb1_dra", "cweb1_ddec",
            "cweb2_dra", "cweb2_ddec"]:
    if col not in global_cat.colnames:
        global_cat[col] = 0.0 * u.arcsec

global_cat

Load the necessary data files

In [ ]:

# Specify the path to the cutouts directory
my_cutouts = '/home/bpc/University/master/Red_Cardinal/cutouts/'
co_rotated = '/home/bpc/University/master/Red_Cardinal/cutouts_rotated/'

# Load the PRIMER cutouts
primer003_rot = glob.glob(os.path.join(co_rotated, "*primer003_rot.fits"))
print(f"Found {len(primer003_rot)} FITS files for the PRIMER survey.")

primer004_rot = glob.glob(os.path.join(co_rotated, "*primer004_rot.fits"))
print(f"Found {len(primer004_rot)} FITS files for the PRIMER survey.")

# Load the COSMOS-Web cutouts
cweb1_rot = glob.glob(os.path.join(co_rotated, "*cweb1_rot.fits"))
print(f"Found {len(cweb1_rot)} FITS files for the COSMOS-Web survey #1.")

cweb2_rot = glob.glob(os.path.join(co_rotated, "*cweb2_rot.fits"))
print(f"Found {len(cweb2_rot)} FITS files for the COSMOS-Web survey #2.")


In [ ]:
compute_offset(co_rotated, 'primer', 'F770W', '003')
compute_offset(co_rotated, 'primer', 'F770W', '004')
compute_offset(co_rotated, survey='cweb', filter='F770W', obs="1")
compute_offset(co_rotated, survey='cweb', filter='F770W', obs="2")

Now we visualise the catalogue and store it in a csv-file

In [ ]:
global_cat
fname = "/home/bpc/University/master/Red_Cardinal/astrometric_offsets.csv"
global_cat.write(fname, format="csv", overwrite=True)

Now that the catalogue table is stored there is no need to rerun the entire code, it can just be loaded into memory

Now let's plot the offset data!

In [ ]:

# Read in the data as a pandas dataframe
fname = "/home/bpc/University/master/Red_Cardinal/astrometric_offsets.csv"
df = pd.read_csv(fname)

# Exclude specific galaxies
exclude_ids = ['19098','19681','21451','7934','8465','9517','10415','11247','11451','12133',
               '12175','12213','7696','9809','10600','11137','16615','16874','17517','11481',
               '12443','20720','21472','21547','22606']

exclude_ids = [int(numeric_string) for numeric_string in exclude_ids]

print(f"Catalogue length: {len(df)}")

# Filter out rows based on exclude_ids
exclude_indices = df[df['id'].isin(exclude_ids)].index
df = df.drop(exclude_indices)

# Check how many rows remain after filtering
print(f"Filtered catalogue length: {len(df)}")

offset_cols = ['primer003_dra', 'primer004_dra', 'cweb1_dra', 'cweb2_dra']

for col1 in offset_cols:
    col2 = col1.replace("dra", "ddec")  # Find corresponding ddec column

    # Keep only rows where `col1` is NOT 0.0
    zero_indices = df[df[col1] == 0.0].index
    df_new = df.drop(zero_indices)

    # Determine survey name
    survey = 'PRIMER' if 'primer' in col1 else 'COSMOS-Web'

    # Scatter plot
    plt.scatter(df_new[col1], df_new[col2])
    plt.xlabel('Delta RA (arcsec)')
    plt.ylabel('Delta Dec (arcsec)')
    plt.title(f'Astrometric offset from {survey} F770W MIRI cutout to F444W NIRCam cutout')
    plt.show()

    # Quiver plot
    fig, ax = plt.subplots()
    
    # Extract filtered values
    dra_values = df_new[col1]
    ddec_values = df_new[col2]
    
    ax.quiver(df_new['ra'], df_new['dec'], dra_values, ddec_values, angles='xy', scale_units='xy', scale=1)
    ax.set(xlabel='RA', ylabel='Dec', title=f'Astrometric offset from {survey} F770W MIRI cutout to F444W NIRCam cutout')
    
    mean_ra = np.mean(df_new['ra'])
    mean_dec = np.mean(df_new['dec'])
    
    # Calculate the limits of the plot based on the data
    ra_min, ra_max = df_new['ra'].min(), df_new['ra'].max()
    dec_min, dec_max = df_new['dec'].min(), df_new['dec'].max()

    # Calculate the arrow lengths to expand the limits accordingly
    arrow_lengths = np.sqrt(df_new[col1]**2 + df_new[col2]**2)
    arrow_min, arrow_max = arrow_lengths.min(), arrow_lengths.max()

    # Extend the axes limits a bit to ensure arrows fit inside the plot window
    ax.set_xlim(ra_min - arrow_max, ra_max + arrow_max)
    ax.set_ylim(dec_min - arrow_max, dec_max + arrow_max)
    plt.show()
